In [1]:
import os
import re
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_community.llms import YandexGPT
from langchain_core.prompts import PromptTemplate

load_dotenv();

## Load data

In [2]:
interviews = {}
files = [file for file in os.listdir("../data/raw/") if file.endswith(".txt")]
for file in files:
    with open(f"../data/raw/{file}", "r") as f:
        interviews[file.split(".")[0]] = f.read()

interviews = dict(sorted(interviews.items()))

In [71]:
for key, val in interviews.items():
    print(f"{key}, len: {len(val)}")

interview_1, len: 36019
interview_2, len: 33945
interview_3, len: 37991
interview_4, len: 39345
interview_5, len: 50578


In [72]:
test_text = interviews["interview_1"]

## Prompts

In [92]:
template_text = """
Твоя задача структурировать стенограмму интервью риелтора и:
1. Выделить основные разделы или фазы интервью (например, введение, дополнительные вопросы, основные темы, заключительные замечания);
2. Разбить стенограмму на эти разделы, отметив начало и конец каждой части.
3. Создайте стандартизированный шаблон для последовательного структурирования стенограмм.

{text}
"""

template_prompt = PromptTemplate(
    template=template_text,
    input_variables=["text"]
)

prompt_structure = template_prompt.format(text=test_text)

In [99]:
template_text = """
Твоя задача выполнить кодирование текста:
1. Внимательно прочитайте стенограмму интервью с риелтором и выделите ключевые фразы, слова или предложения, которые отражают важные идеи, концепции или опыт.
2. Присвой выделенным фрагментам предварительные коды или ярлыки, используя короткие описательные фразы или слова, которые обобщают основную идею.
3. Отдельным выведи созданный тобой список кодов/ярлыков для их дальнейшего применения в анализе других стенограмм интервью с риелторами.

{text}
"""

template_prompt = PromptTemplate(
    template=template_text,
    input_variables=["text"]
)

prompt_code = template_prompt.format(text=test_text)

## Yandex GPT
- langchain [doc](https://api.python.langchain.com/en/latest/llms/langchain_community.llms.yandex.YandexGPT.html#langchain_community.llms.yandex.YandexGPT) 

In [74]:
YANDEX_FOLDER_ID = os.getenv("YANDEX_FOLDER_ID")
YANDEX_API_KEY = os.getenv("YANDEX_API_KEY")

llm_yandex = YandexGPT(api_key=YANDEX_API_KEY, folder_id=YANDEX_FOLDER_ID, model_name="summarization", temperature=0.0)

In [95]:
def get_num_tokens(*args):
    for arg in args:
        print(llm_yandex.get_num_tokens(arg))

In [97]:
get_num_tokens(prompt_structure, prompt_code)

39242
39384


### Map Reduce - Summarize a couple pages multiple pages

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [114]:
# summary_chain = load_summarize_chain(llm=llm_yandex, chain_type='map_reduce', verbose=True)
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=6000, chunk_overlap=500)
docs = text_splitter.create_documents([test_text])

In [101]:
map_prompt = """
Твоя задача структурировать стенограмму интервью риелтора:
"{text}"
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

combine_prompt = """
Напишите краткое изложение следующего текста, разделенного тройными обратными кавычками:
1. Выделить основные разделы или фазы интервью (например, введение, дополнительные вопросы, основные темы, заключительные замечания);
2. Разбить стенограмму на эти разделы, отметив начало и конец каждой части.
3. Создайте стандартизированный шаблон для последовательного структурирования стенограмм.

```{text}```
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

summary_chain = load_summarize_chain(llm=llm_yandex,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
                                     verbose=True)

In [105]:
# output = summary_chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Твоя задача структурировать стенограмму интервью риелтора:
"Запись, наши с вами обсуждения.

Смотрите, давайте тогда с самого начала, вот напомним, просто, чтобы у нас человек, который будет расшифровывать, чтобы он не бегал там по разным записям, да?

Расскажите, пожалуйста, еще раз немного о себе.

Сколько вам лет, где живете и как давно работаете риэлтором?

42 года, живу в Москве, работаю порядка, ну, мне сейчас сложно сказать, но можно сказать, там 5-7 лет, но 5 - это вот так плотно.

Работаю сама по себе, сама на себя, не в агентстве, частный риэлтор.

Так, отлично.

Расскажите, пожалуйста, вообще, как пришли к пониманию, что хотите заниматься, ну, скажем так, именно частным риэлторством, да, а не пойти, например, в агентство работать?

Началось с того, что нужно было продать свою квартиру, поэтому погрузилась в изучение данной процедуры, там договора, все дела, оформлен

In [106]:
# print(output)

- Интервью с Леонором Козлов, риэлтором в Москве.
- Леонора начала работу в частном риэлтерстве 5-7 лет назад, занималась вторичным жильем.
- Занимается страхованием и другими услугами самостоятельно.
- Рекомендует начать карьеру в недвижимости через холдинги и литературу.
- Нежелание заниматься риэлторской деятельностью из-за нехватки ресурсов и отсутствия интереса.
- Продажа первичного жилья осложнена законодательством, покупка без риэлтора в России невозможна.
- Риэлторы должны обращаться к аккредитованным людям для сделок и выбирать курсы обучения.
- Оптимальный формат курсов: онлайн и оффлайн, индивидуальное наставничество.


In [110]:
map_prompt = """
Твоя задача выполнить кодирование текста, разделенного тройными обратными кавычками:
1. Внимательно прочитайте стенограмму интервью с риелтором и выделите ключевые фразы, слова или предложения, которые отражают важные идеи, концепции или опыт.
2. Присвой выделенным фрагментам предварительные коды или ярлыки, используя короткие описательные фразы или слова, которые обобщают основную идею.
3. Отдельным выведи созданный тобой список кодов/ярлыков для их дальнейшего применения в анализе других стенограмм интервью с риелторами.

```{text}```
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

# combine_prompt = """
# Твоя задача выполнить кодирование текста, разделенного тройными обратными кавычками:
# 1. Внимательно прочитайте стенограмму интервью с риелтором и выделите ключевые фразы, слова или предложения, которые отражают важные идеи, концепции или опыт.
# 2. Присвой выделенным фрагментам предварительные коды или ярлыки, используя короткие описательные фразы или слова, которые обобщают основную идею.
# 3. Отдельным выведи созданный тобой список кодов/ярлыков для их дальнейшего применения в анализе других стенограмм интервью с риелторами.
# 
# ```{text}```
# """
# combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

summary_chain_code = load_summarize_chain(llm=llm_yandex,
                                          chain_type='map_reduce',
                                          map_prompt=map_prompt_template,
                                          # combine_prompt=combine_prompt_template,
                                          verbose=True)

# output_code = summary_chain_code.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Твоя задача выполнить кодирование текста, разделенного тройными обратными кавычками:
1. Внимательно прочитайте стенограмму интервью с риелтором и выделите ключевые фразы, слова или предложения, которые отражают важные идеи, концепции или опыт.
2. Присвой выделенным фрагментам предварительные коды или ярлыки, используя короткие описательные фразы или слова, которые обобщают основную идею.
3. Отдельным выведи созданный тобой список кодов/ярлыков для их дальнейшего применения в анализе других стенограмм интервью с риелторами.

```Запись, наши с вами обсуждения.

Смотрите, давайте тогда с самого начала, вот напомним, просто, чтобы у нас человек, который будет расшифровывать, чтобы он не бегал там по разным записям, да?

Расскажите, пожалуйста, еще раз немного о себе.

Сколько вам лет, где живете и как давно работаете риэлтором?

42 года, живу в Москве, работаю порядка, ну, мне сей

Retrying langchain_community.llms.yandex.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {created_time:"2024-05-05T13:59:39.673924+03:00", grpc_status:8, grpc_message:"ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"}"
>.



> Finished chain.

> Finished chain.


In [112]:
print(output_code)

- Задача: анализ опыта работы риелторов через кодирование их интервью.
- Ключевая информация: опыт работы с разными типами сделок, личный опыт.
- Составление списка кодов для кодирования интервью.
- Сложности в поиске информации, выборе обучения, критерии выбора.
- Выбор между блогерами, агентами и курсами по недвижимости.
- Формат курсов: онлайн, оффлайн, вебинары с обратной связью.
- Оптимальное количество участников, материалы после курсов.
- Риски предоставления доступа к материалам, система домашних заданий.
- Тенденции рынка недвижимости, удобство звонков, обратная связь, кураторство.
- Продолжительность занятий, платные/бесплатные курсы.
- Аспекты обучения: hard- и soft-навыки, баланс между ними.
- Недостатки каналов Telegram и соцсетей.
- Оценка журнала "Метр Квадратный" и его тем.


In [ ]:
map_prompt = """
Твоя задача проанализировать стенограмму интервью риелтора, разделенного тройными обратными кавычками и :
Просмотрите исходные коды, которые вы разработали, и найдите среди них закономерности, сходства
или взаимосвязи.
○ Сгруппируйте соответствующие коды в более широкие категории или темы, которые
охватывают основные темы, обсуждавшиеся в интервью.
○ Определите все подтемы в рамках каждой основной темы, чтобы обеспечить более
детальное понимание тем.
○ Создайте визуальное представление (например, карту памяти, тематическую сеть), чтобы
проиллюстрировать взаимосвязи между темами и подтемами.

```{text}```
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

# combine_prompt = """
# Твоя задача выполнить кодирование текста, разделенного тройными обратными кавычками:
# 1. Внимательно прочитайте стенограмму интервью с риелтором и выделите ключевые фразы, слова или предложения, которые отражают важные идеи, концепции или опыт.
# 2. Присвой выделенным фрагментам предварительные коды или ярлыки, используя короткие описательные фразы или слова, которые обобщают основную идею.
# 3. Отдельным выведи созданный тобой список кодов/ярлыков для их дальнейшего применения в анализе других стенограмм интервью с риелторами.
# 
# ```{text}```
# """
# combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

summary_chain_topics = load_summarize_chain(llm=llm_yandex,
                                            chain_type='map_reduce',
                                            map_prompt=map_prompt_template,
                                            # combine_prompt=combine_prompt_template,
                                            verbose=True)

# output_code = summary_chain_code.run(docs)

# --−    Structuring the transcripts    ---

In [121]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

In [128]:
# Map
map_template = """
Ниже представлен пакет документов относящихся к одному интервью ритейл-специалиста.

"{docs}"

На основании этого списка документов: 
1. Выделить основные разделы или фазы интервью (например, введение, дополнительные вопросы, основные темы, заключительные замечания);
2. Разбить стенограмму на эти разделы, отметив начало и конец каждой части.
3. Создайте стандартизированный шаблон для последовательного структурирования стенограмм.
"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm_yandex, prompt=map_prompt)

In [129]:
from langchain import hub

# map_prompt = hub.pull("rlm/map-prompt")
# map_chain = LLMChain(llm=llm_yandex, prompt=map_prompt)

In [130]:
reduce_template = """
Ниже представлен набор резюме:

"{docs}"

Возьми их и преврати в окончательное, сводное изложение основных тем. 
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [131]:
# reduce_prompt = hub.pull("rlm/map-prompt")

In [133]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [134]:
# Run chain
reduce_chain = LLMChain(llm=llm_yandex, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="docs")

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=7000,
)

In [138]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=6000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [139]:
output = map_reduce_chain.run(split_docs)

In [141]:
print(output)

- Интервью с ретейлом специалистом, обсуждающий свой опыт работы в частном риэлторстве и изучение процедур оформления сделок
- Сравнивает работу с агентствами и подход к обучению
- Обеспокоен сложностью работы в одиночку и ищет способы обучения
- Изучает литературу и журналы, рассматривает сотрудничество с застройщиками
- Подчеркивает важность опыта и практических навыков в обучении
- Размышляет о выборе курсов и их содержании, сравнивает онлайн и офлайн форматы
- Предлагает разбить курсы на части для эффективного обучения, предпочитает групповые занятия перед индивидуальными
- Обсуждается организация курсов, обратная связь и хранение материалов
- Высказывается о важности чатов для выявления запросов аудитории
- Отмечает важность сертификатов и баланса между hard и soft навыками
- Анализирует каналы получения информации, предлагает изменения в журнале "Метр квадратный"


# --−    Performing initial coding    ---

In [142]:
# Map
map_template = """
Ниже представлен пакет документов относящихся к одному интервью ритейл-специалиста.

"{docs}"

На основании этого списка документов: 
1. Выделите ключевые фразы, слова или предложения, которые отражают важные идеи, концепции или опыт.
2. Присвой выделенным фрагментам предварительные коды или ярлыки, используя короткие описательные фразы или слова, которые обобщают основную идею.
3. Отдельным выпиши созданный тобой список кодов/ярлыков для их дальнейшего применения в анализе других стенограмм интервью с риелторами.
"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm_yandex, prompt=map_prompt)

In [143]:
reduce_template = """
Ниже представлен набор резюме:

"{docs}"

Возьми их и преврати в окончательное, сводное изложение основных тем. 
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [144]:
# Run chain
reduce_chain = LLMChain(llm=llm_yandex, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="docs")

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=7000,
)

In [145]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=6000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [146]:
output_code = map_reduce_chain.run(split_docs)

In [147]:
print(output_code)

- Интервью с риэлтором, работавшим частным риэлтором
- Основные направления работы: вторичное жилье
- Проблемы отсутствия посредника при оформлении сделки
- Обязанности: поиск объектов, оформление документов, консультации
- Помощь от соседки для освоения нюансов работы
- Холдинги White предлагают хорошую систему продажи и преференций на первичном рынке
- Обсуждение обучения и развития в сфере недвижимости
- Возможность упаковать себя в качестве риелдера
- Работа в агентстве предпочтительнее, чем позиционирование себя как одиночного специалиста
- Исследование литературы и журналов для восполнения навыков и знаний
- Различие между частным риэлтером и агентством
- Сложности в поиске информации для повышения экспертной оценки
- Вопросы о критериях и содержании курсов обучения
- Изучение создателя и школы курса
- Ценность оцифрованного формата курса
- Онлайн- и оффлайн-форматы курсов
- Вебинары и обратная связь важны для эффективного обучения
- Ограничение участников группы до 10 для эффект

# --−   Calculating the frequency of problems mentioned     ---

In [148]:
# Map
map_template = """
Ниже представлен пакет документов относящихся к одному интервью ритейл-специалиста.

"{docs}"

На основании этого списка документов подсчитайте частоту упоминания проблем.:
1. Составь список всех конкретных проблем, упомянутых респондентами.
2. Просмотрите каждую стенограмму и подсчитайте, сколько раз упоминалась каждая проблема.
3. Рассчитайте частоту встречаемости каждой проблемы, разделив количество упоминаний на общее количество интервью.
4. Создайте таблицу или диаграмму, чтобы отобразить частоту встречаемости каждой проблемы в порядке убывания.

"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm_yandex, prompt=map_prompt)

In [149]:
reduce_template = """
Ниже представлен набор резюме:

"{docs}"

Возьми их и преврати в окончательное, сводное изложение основных тем. 
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [150]:
# Run chain
reduce_chain = LLMChain(llm=llm_yandex, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="docs")

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=7000,
)

In [151]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=6000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [152]:
output_cnt_problem = map_reduce_chain.run(split_docs)

In [153]:
print(output_cnt_problem)

- Интервью с риэлтером: возраст 42, работает 5-7 лет, ушла из агентства из-за плохого подхода и отсутствия нацеленности на клиентов. Основные сделки - вторичное жилье, обязанности: поиск объектов и оформление документов. Не проходила обучение, рекомендует крупные холдинги.
- Ритейл-специалист обсуждает обучение и развитие, нежелание позиционировать себя, просмотр литературы и форумов, сравнение работы частного риэлтера и агентства.
- Обсуждаются критерии выбора курса, цена и организатор, обратная связь и консультации, онлайн и оффлайн форматы, вебинары, темы, требующие разбиения на мелкие составляющие, домашние задания и проверки знаний, форматы заданий, обратная связь от клиентов, возможные способы обратной связи.
- Чаты и их ценность, продолжительность занятий, платные и бесплатные курсы, полезность и бесполезность обучения, важность soft- и hard-skils, баланс между hard- and soft-skills, каналы получения информации, недостаток каналов, проблемы журналов о недвижимости, необходимость